# L1: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.
Six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai crewai_tools langchain_community
```

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [5]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ["OPENAI_KEY"]
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [8]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [9]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [10]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [11]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [12]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [15]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 
[2025-03-14 22:51:59][🚀 CREW 'CREW' STARTED, 3D7701C8-759E-4897-ACD4-BF1A3A583BCE]: 2025-03-14 22:51:59.793517
 
[2025-03-14 22:51:59][📋 TASK STARTED: 1. PRIORITIZE THE LATEST TRENDS, KEY PLAYERS, AND NOTEWORTHY NEWS ON ARTIFICIAL INTELLIGENCE.
2. IDENTIFY THE TARGET AUDIENCE, CONSIDERING THEIR INTERESTS AND PAIN POINTS.
3. DEVELOP A DETAILED CONTENT OUTLINE INCLUDING AN INTRODUCTION, KEY POINTS, AND A CALL TO ACTION.
4. INCLUDE SEO KEYWORDS AND RELEVANT DATA OR SOURCES.]: 2025-03-14 22:51:59.805033
 
[2025-03-14 22:51:59][🤖 AGENT 'CONTENT PLANNER' STARTED TASK]: 2025-03-14 22:51:59.806033
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 
[2025-03-14 22:51:59][🤖 LLM CALL STARTED

- Display the results of your execution as markdown in the notebook.

In [19]:
from IPython.display import Markdown

# Assuming `result` is an instance of `CrewOutput`
Markdown(result.raw)

```markdown
# Unlocking the Future: How Artificial Intelligence is Shaping Our World

## Introduction
Artificial Intelligence (AI) is no longer a concept confined to science fiction; it has rapidly transformed into a multifaceted realm influencing numerous sectors. As AI technologies evolve, understanding their implications becomes vital for individuals and organizations alike. With advancements occurring at a breakneck pace, the importance of staying informed about the latest trends in AI cannot be overstated.

In this blog post, we will delve into the most significant developments in AI, explore the key players who are driving innovation, and discuss the ethical considerations that accompany these advancements. Join us on this journey as we uncover how artificial intelligence is shaping our world today and in the future.

## What is Artificial Intelligence?
Artificial Intelligence encompasses a wide variety of technologies designed to simulate human intelligence. From machine learning algorithms that learn from data to natural language processing (NLP) systems that facilitate human-machine communication, AI represents an array of processes that can mimic cognitive functions. The field has its roots in the 1950s, with early concepts set forth by pioneers like Alan Turing and John McCarthy.

Over the years, AI has evolved dramatically, now encompassing deep learning, computer vision, and robotics. Its application has expanded well beyond academia, now permeating industries, businesses, and even daily life, thus reshaping how we interact with technology.

## Latest Trends in AI
### Generative AI
One of the most exciting trends in artificial intelligence is generative AI, exemplified by models like GPT-4, Midjourney, and DALL-E. These AI systems possess the unique ability to create original content, including text, images, and music. As such, they illustrate the innovative capabilities of AI, pushing the boundaries of creativity and innovation.

### AI in Healthcare
Another vital area is AI's integration into healthcare, where it is revolutionizing diagnostics and personalized medicine. Algorithms powered by AI can analyze medical data faster and more accurately than human professionals, helping healthcare providers design treatment plans tailored to individual patients. The potential benefits range from improved patient care to enhanced operational efficiency, underscoring AI's critical role in this essential sector.

## Key Players in the AI Space
### Tech Giants Leading the Charge
Among the leaders in the AI landscape are tech giants like Google, Microsoft, OpenAI, and IBM, each contributing to transformative AI developments. Google’s DeepMind has set new benchmarks in machine learning, while OpenAI has made waves with its user-friendly technologies that assist in various applications. These innovations from established companies foster advancements that have widespread implications across numerous fields.

### Innovative Startups and Research Institutions
In addition to established names, a host of innovative startups such as Hugging Face and UiPath are making noteworthy strides in AI applications tailored for businesses. Coupled with research institutions like Stanford and MIT, which continuously push the boundaries of AI exploration, this dynamic environment signifies vast potential for breakthroughs and contributions that will influence future technology directions.

## AI Ethical Considerations
As AI increasingly influences various sectors, discussions on ethics, data privacy, and algorithmic bias have come to the forefront. Major stakeholders recognize the significance of developing ethical frameworks that protect individuals and ensure fairness within AI systems. The conversation surrounding AI regulations, notably in the European Union, indicates a critical shift toward accountability and responsible AI implementation.

Respecting user privacy and addressing bias in data sets are paramount for organizations aiming to build trust with their customers. As AI evolves, striking a balance between innovation and ethics will be crucial for success.

## Real-World Applications of AI
AI's impact transcends theoretical discussions and spills into tangible outcomes across sectors. In healthcare, AI tools are being used for predictive analytics, improving patient outcomes with proactive solutions. In finance, AI systems like credit scoring algorithms enable more accurate risk assessments, while retail industries utilize AI for personalized shopping experiences.

Entertainment also benefits from AI advancements, as algorithms curate content based on user preferences and viewing habits. The diversity of applications illustrates the far-reaching implications of AI in enhancing efficiency and transforming user experiences across multiple domains.

## Future Perspectives
Looking ahead, the future of artificial intelligence is laden with possibilities. As technologies evolve, we can expect to see advancements in areas such as emotion recognition and advanced reasoning systems. The potential integration of AI into fields like autonomous driving and smart city innovations represents just the tip of the iceberg.

However, just as innovation offers boundless opportunities, it also presents challenges that must be navigated thoughtfully. Ensuring that AI developments align with societal values and ethical standards will be imperative for fostering positive outcomes.

## Conclusion
The journey to understand artificial intelligence is ongoing, with each development revealing new possibilities and challenges. From generative AI innovations to ethical discussions and real-world applications, the landscape of AI is ever-evolving. As stakeholders from various domains navigate this intricate environment, staying informed about AI advancements is crucial.

By engaging in dialogue surrounding AI's growth and advocating for responsible practices, we can collectively ensure that its influence leads to positive societal change. I encourage readers to subscribe for updates, share their thoughts, and actively participate in discussions about AI and its profound implications for our world.

---
```

This blog post is structured to engage readers while providing informative insights into artificial intelligence, emphasizing the key points outlined in the content plan and incorporating relevant SEO keywords seamlessly.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).